In [11]:
# Question 1: What is the most popular attraction to visit in the park?

import sqlite3

db_connect = 'dinofunworld.db'
conn = sqlite3.connect(db_connect)
cursor = conn.cursor()

cursor.execute("""
    SELECT Name 
    FROM (
        SELECT COUNT(*) as total, attraction 
        FROM checkin 
        GROUP BY attraction 
        ORDER BY total DESC 
        LIMIT 1
    ) a, attraction b 
    WHERE a.attraction = b.AttractionID
""")

result = cursor.fetchone()

if result:
    print(result[0])

conn.close()

Atmosfear


In [12]:
# Question 2: What ride (note that not all attractions are rides) has the longest visit time?

import sqlite3
from datetime import datetime, timedelta

# Connect to the SQLite database
db_connect = 'dinofunworld.db'
conn = sqlite3.connect(db_connect)
cursor = conn.cursor()

# Execute the SQL queries
cursor.execute("SELECT attraction, duration FROM checkin WHERE attraction IN (SELECT AttractionID FROM attraction WHERE Category LIKE '%Rides%') AND duration NOT LIKE '%@SD' AND duration IS NOT NULL")
rows = cursor.fetchall()

# Convert duration to seconds and calculate total duration and visit count for each attraction
attraction_duration = {}
attraction_count = {}
for row in rows:
    attraction = row[0]
    duration = row[1]
    if duration is not None and '@SD' not in duration:
        t = datetime.strptime(duration, "%H:%M:%S")
        delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
        total_seconds = delta.total_seconds()
        if attraction in attraction_duration:
            attraction_duration[attraction] += total_seconds
            attraction_count[attraction] += 1
        else:
            attraction_duration[attraction] = total_seconds
            attraction_count[attraction] = 1

# Calculate average duration for each attraction
attraction_avg = {attraction: round(attraction_duration[attraction] / attraction_count[attraction], 2) for attraction in attraction_duration}

# Find the attraction with the maximum average duration
max_avg_attraction = max(attraction_avg, key=attraction_avg.get)

# Get the name of the attraction with the maximum average duration
cursor.execute("SELECT Name FROM attraction WHERE AttractionID = ?", (max_avg_attraction,))
result = cursor.fetchone()

# Print the result
if result:
    print(result[0])

# Close the connection
conn.close()

TerrorSaur


In [13]:
# Question 3: Which Fast Food offering has the fewest visitors?

import sqlite3

# Connect to the SQLite database
db_connect = 'dinofunworld.db'
conn = sqlite3.connect(db_connect)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
    SELECT b.Name 
    FROM (
        SELECT attraction, COUNT(*) as total 
        FROM checkin  
        WHERE attraction IN (
            SELECT AttractionID  
            FROM attraction 
            WHERE type LIKE '%Fast %'
        ) 
        GROUP BY attraction 
        ORDER BY total 
        LIMIT 1
    ) a, attraction b 
    WHERE a.attraction = b.AttractionID
""")

# Fetch the result
result = cursor.fetchone()

# Print the result
if result:
    print(result[0])

# Close the connection
conn.close()

Theresaur Food Stop


In [14]:
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

import sqlite3,pandas
from datetime import datetime, timedelta

# Connect to the SQLite database
db_connect='dinofunworld.db'
conn=sqlite3.connect(db_connect)

# Create a cursor object using the cursor() method
cursor=conn.cursor()

# Do the query using pandas
df=pandas.read_sql_query("select attraction,duration from checkin where  attraction in (select AttractionID from attraction where Category like '%Rides%') and duration not like '%@SD' and duration is not null",conn)
df1=pandas.read_sql_query("select count(*) as Totalvisits,attraction  from checkin where  attraction in (select AttractionID from attraction where Category like '%Rides%') and duration not like '%@SD%' and duration is not null group by attraction",conn)
df3=pandas.merge(df, df1, on='attraction')

def convert_to_datetime(time):
    t = datetime.strptime(time,"%H:%M:%S")
    delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
    return delta.total_seconds()

df['duration'] = df['duration'].apply(convert_to_datetime)
newdf=df.groupby(['attraction']).sum()
new1df=pandas.merge(newdf, df1, on='attraction')
new1df['avg']=round(new1df['duration']/new1df['Totalvisits'],2)
result=new1df.loc[new1df['avg'].idxmax()]
attractiondf=pandas.read_sql_query("select Name,AttractionID from attraction where Category like '%Rides%' ",conn)
listatt=attractiondf.values.tolist()
res = [item for item in listatt
  if item[1] == result.attraction]
res[0][0]
list=new1df.values.tolist()
pd=pandas.DataFrame(columns=['attraction'])
finalresult=[]
for  tuple in enumerate(list):
    count=0
    for  another in enumerate(list):
       if tuple[1][2]<another[1][2] and tuple[1][3]>another[1][3]:
         count=count+1
    if count == 0:
     pd=tuple[1]
     res = [item for item in listatt
      if item[1] == pd[0]]
     finalresult.append(res[0][0])

finalresult

['Wrightiraptor Mountain',
 'Galactosaurus Rage',
 'Keimosaurus Big Spin',
 'Atmosfear',
 'Dykesadactyl Thrill']